<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/step2_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyreadr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 13.1 MB/s eta 0:00:00


In [8]:
# 启用 R 魔法
%load_ext rpy2.ipython


In [11]:
%%R

# 使用 R 语言读取 .rds 文件并写为 .csv
rds_path <- "/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.new2023.rds"
df <- readRDS(rds_path)

# 保存为 csv
write.csv(df, "/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.new2023.csv", row.names = FALSE)


# 数据预处理

In [13]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.new2023.csv", encoding='latin1')
df.head()


,FoodNumber,FoodName,SubFoodGroupCode,SubFoodGroupDesc,RecipeName,RecipeSubFoodGroupCode,RecipeSubFoodGroupDesc,NOVA
0,8112,TURMERIC GROUND,50R,SAVOURY SAUCES PICKLES GRAVIES & CONDIMENTS,NaN,25B,OTHER PORK INCLUDING HOMEMADE RECIPE DISHES,4
1,10701,HALIBORANGE KIDS MULTIVITAMIN FRUIT SOFTIES,54G,VITAMINS (TWO OR MORE INCL MULTIVITS) NO MINE...,NaN,54G,VITAMINS (TWO OR MORE INCL MULTIVITS) NO MINE...,NC
2,8694,TURKEY AND PORK LUNCHEON MEAT EG. SAINSBURYS B...,32A,OTHER MEAT PRODUCTS MANUFACTURED INCL READY M...,NaN,32A,OTHER MEAT PRODUCTS MANUFACTURED INCL READY M...,4
3,2522,SALT TABLE,50R,SAVOURY SAUCES PICKLES GRAVIES & CONDIMENTS,NaN,39A,OTHER POTATO PRODUCTS & DISHES - MANUFACTURED,3
4,10078,"BASSETTS SOFT AND CHEWY VITAMINS A,C,D,E",54G,VITAMINS (TWO OR MORE INCL MULTIVITS) NO MINE...,NaN,54G,VITAMINS (TWO OR MORE INCL MULTIVITS) NO MINE...,NC


# 第二步：标准清洗建议（按你的以往做法）
1. 统一小写和空格处理（用于后续匹配）

In [25]:
df['FoodName_clean'] = df['FoodName'].str.lower().str.strip().str.replace(r"\s+", " ", regex=True)
df['SubFoodGroupDesc_clean'] = df['SubFoodGroupDesc'].str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# 2. 检查是否有缺失 NOVA 值（如你要补全的部分）
df['NOVA'].isna().sum(), df['NOVA'].value_counts()

# 保存清洗后的版本备用：
df.to_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.cleaned.csv", index=False)


# 读取文件（如果你之前还没读取）
df = pd.read_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.cleaned.csv", encoding="latin1")

# 筛选你想保留的列（原始group列 + 清洗后的clean列 + NOVA）
cols_to_keep = [
    'FoodNumber', 'FoodName_clean', 'SubFoodGroupDesc_clean', 'RecipeSubFoodGroupCode',
    'NOVA'
]

# 保留这些列
df_cleaned = df[cols_to_keep]

# 保存为一个新的简洁文件
df_cleaned.to_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.cleaned_slim.csv", index=False)


# 匹配

In [29]:
# ✅ 1. 读取文件
df_review = pd.read_excel("/content/drive/MyDrive/UPF-HFI/0728outcome/step2_needs_manual_review.xlsx")
df_nova11 = pd.read_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.cleaned.csv", encoding="latin1")

# ✅ 2. 清洗字段（只保留匹配用字段）
df_review['Foodgroupen_clean'] = df_review['Foodgroupen'].str.lower().str.strip().str.replace(r"\s+", " ", regex=True)
df_nova11['SubFoodGroupDesc_clean'] = df_nova11['SubFoodGroupDesc'].str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# ✅ 3. 执行匹配（只对 * 行匹配 group）
df_merge = df_review.merge(
    df_nova11[['SubFoodGroupDesc_clean', 'NOVA']],
    how='left',
    left_on='Foodgroupen_clean',
    right_on='SubFoodGroupDesc_clean'
)

# ✅ 4. 更新结果
df_merge['NOVA_step2'] = df_merge['NOVA']
df_merge['match_reason_step2'] = df_merge['NOVA'].apply(
    lambda x: 'matched from Year11 by SubFoodGroupDesc_clean' if pd.notna(x) else None
)

# ✅ 5. 保存输出（含 step2 匹配结果）
df_merge.to_excel("/content/drive/MyDrive/UPF-HFI/0728outcome/step2_output_group_only.xlsx", index=False)
